# 5.3 Lab: Cross-Validation and the Bootstrap

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import rpy2.robjects as robjects
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri


%load_ext rpy2.ipython

## 5.3.1 The Validation Set Approach

In [2]:
%%R
library(ISLR2)
set.seed(1)
train <- sort(sample(392, 196))

In [3]:
%%R
train

  [1]   1   9  13  14  15  16  17  19  20  22  24  25  29  30  31  33  36  37
 [19]  39  40  41  42  43  44  45  48  49  50  51  53  61  64  65  70  72  75
 [37]  77  78  79  80  84  85  86  88  89  92  93  98 102 103 104 105 106 107
 [55] 108 110 111 116 117 118 121 122 124 126 127 129 130 133 135 138 140 141
 [73] 143 145 149 152 157 159 160 163 164 165 167 170 172 174 176 181 185 187
 [91] 193 194 195 198 201 204 206 207 212 213 214 217 218 219 220 221 223 224
[109] 225 228 229 230 233 234 237 238 239 240 242 244 246 247 248 252 255 258
[127] 263 270 271 272 277 279 280 281 282 284 285 287 289 290 294 295 296 298
[145] 299 304 306 307 309 310 313 315 318 319 323 324 325 326 328 329 330 331
[163] 337 340 341 342 343 344 347 348 349 350 355 358 362 363 364 366 367 368
[181] 369 372 373 374 375 376 378 382 383 384 386 388 389 390 391 392


In [4]:
data = robjects.r("""
library(ISLR2)
set.seed(1)
train <- sample(392, 196)
""")

train_idx = np.array(data)
train_idx = np.sort(train_idx)

In [5]:
train_idx

array([  1,   9,  13,  14,  15,  16,  17,  19,  20,  22,  24,  25,  29,
        30,  31,  33,  36,  37,  39,  40,  41,  42,  43,  44,  45,  48,
        49,  50,  51,  53,  61,  64,  65,  70,  72,  75,  77,  78,  79,
        80,  84,  85,  86,  88,  89,  92,  93,  98, 102, 103, 104, 105,
       106, 107, 108, 110, 111, 116, 117, 118, 121, 122, 124, 126, 127,
       129, 130, 133, 135, 138, 140, 141, 143, 145, 149, 152, 157, 159,
       160, 163, 164, 165, 167, 170, 172, 174, 176, 181, 185, 187, 193,
       194, 195, 198, 201, 204, 206, 207, 212, 213, 214, 217, 218, 219,
       220, 221, 223, 224, 225, 228, 229, 230, 233, 234, 237, 238, 239,
       240, 242, 244, 246, 247, 248, 252, 255, 258, 263, 270, 271, 272,
       277, 279, 280, 281, 282, 284, 285, 287, 289, 290, 294, 295, 296,
       298, 299, 304, 306, 307, 309, 310, 313, 315, 318, 319, 323, 324,
       325, 326, 328, 329, 330, 331, 337, 340, 341, 342, 343, 344, 347,
       348, 349, 350, 355, 358, 362, 363, 364, 366, 367, 368, 36

In [6]:
%%R
Auto[train, c(1,9)]

     mpg                              name
326 44.3              vw rabbit c (diesel)
169 23.0                        ford pinto
131 26.0                        ford pinto
301 23.9 oldsmobile cutlass salon brougham
272 23.2                  plymouth sapporo
189 16.0            dodge coronet brougham
309 33.5                   pontiac phoenix
86  13.0                 buick century 350
279 31.5               volkswagen scirocco
367 17.6            chrysler lebaron salon
333 29.8                  vokswagen rabbit
265 18.1                       ford futura
332 33.8                         subaru dl
80  26.0                   renault 12 (sw)
215 13.0                         ford f108
38  18.0                       amc matador
106 13.0            plymouth custom suburb
219 36.0                     renault 5 gtl
371 31.0        pontiac j2000 se hatchback
167 13.0                   ford mustang ii
292 19.2     chevrolet malibu classic (sw)
388 26.0        chrysler lebaron medallion
90  15.0   

In [7]:
import statsmodels.formula.api as smf

In [8]:
auto_df = pd.read_csv("../../../datasets/Auto.csv", na_values='?')

auto_df = auto_df.set_index(keys=np.arange(1, len(auto_df) + 1))

auto_df = auto_df.dropna()

# auto_df = auto_df.set_index(keys=np.arange(1, len(auto_df) + 1))

auto_train_mask = auto_df.index.isin(train_idx)

lm_model = smf.ols(formula = 'mpg ~ horsepower', data = auto_df.iloc[train_idx-1])
lm_fit = lm_model.fit()

In [9]:
auto_df[-1:]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
397,31.0,4,119.0,82.0,2720,19.4,82,1,chevy s-10


In [10]:
auto_df.loc[392]

mpg                         27.0
cylinders                      4
displacement               151.0
horsepower                  90.0
weight                      2950
acceleration                17.3
year                          82
origin                         1
name            chevrolet camaro
Name: 392, dtype: object

In [11]:
auto_df.iloc[train_idx-1]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
9,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
13,15.0,8,400.0,150.0,3761,9.5,70,1,chevrolet monte carlo
14,14.0,8,455.0,225.0,3086,10.0,70,1,buick estate wagon (sw)
15,24.0,4,113.0,95.0,2372,15.0,70,3,toyota corona mark ii
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [12]:
%%R
length(train)

[1] 196


In [13]:
%%R
dim(Auto[train, ])

[1] 196   9


In [14]:
auto_df[auto_train_mask].shape

(191, 9)

In [15]:
%%R
mean(Auto[train, 1])

[1] 23.87449


In [16]:
auto_df[auto_train_mask]['mpg'].mean()

23.82774869109948

In [17]:
%%R
lm.fit <- lm(mpg ~ horsepower, data = Auto, subset = train)

In [18]:
%%R
summary(lm.fit)


Call:
lm(formula = mpg ~ horsepower, data = Auto, subset = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.3177 -3.5428 -0.5591  2.3910 14.6836 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 41.283548   1.044352   39.53   <2e-16 ***
horsepower  -0.169659   0.009556  -17.75   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 5.032 on 194 degrees of freedom
Multiple R-squared:  0.619,	Adjusted R-squared:  0.6171 
F-statistic: 315.2 on 1 and 194 DF,  p-value: < 2.2e-16



In [19]:
lm_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.617
Method:                 Least Squares   F-statistic:                     315.2
Date:                Sat, 21 Jan 2023   Prob (F-statistic):           1.61e-42
Time:                        17:31:13   Log-Likelihood:                -593.80
No. Observations:                 196   AIC:                             1192.
Df Residuals:                     194   BIC:                             1198.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     41.2835      1.044     39.530      0.000      39.224      43.343
horsepower    -0.1697      0.010    -17.754      0.000      -0.189      -0.151
==============================================================================
Omnibus:                       13.451   Durbin-Watson:                   1.171
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.488
Skew:                           0.662   Prob(JB):                     0.000714
Kurtosis:                       3.149   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# because R using integer index labels when doing stuff like Auto[train, ], we need to use .iloc on our df in Python to copy the behavior.  Using .set_index, changes row labels, but not integer labels, and the integer labels always begin at 0, so we use np.arange(0, 392) to produce the indices 0 to 391, for the 392 entries in the auto_df.  Next, because train_idx is the list of integer row labels from R, which begins integer labels at 1, we need to subtract 1 from each entry to match the row integer labels in Python.  Using the set() function allows us to find the set difference, or the integer indices for rows not in our training set.  Because the set difference returns a set, which is treated as a single element, we can't use it with .iloc to get the rows we want, instead we convert the set to a list first.
test_idx = list(set(np.arange(0,392)) - set(train_idx-1))

In [21]:
auto_df.iloc[test_idx]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
2,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
3,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
4,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
5,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
6,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500
...,...,...,...,...,...,...,...,...,...
384,32.0,4,91.0,67.0,1965,15.7,82,3,honda civic (auto)
385,38.0,4,91.0,67.0,1995,16.2,82,3,datsun 310 gx
386,25.0,6,181.0,110.0,2945,16.4,82,1,buick century limited
390,32.0,4,144.0,96.0,2665,13.9,82,3,toyota celica gt


In [22]:
%%R
Auto[-train, c(1,9)]

     mpg                                 name
2   15.0                    buick skylark 320
3   18.0                   plymouth satellite
4   16.0                        amc rebel sst
5   17.0                          ford torino
6   15.0                     ford galaxie 500
7   14.0                     chevrolet impala
8   14.0                    plymouth fury iii
10  15.0                   amc ambassador dpl
11  15.0                  dodge challenger se
12  14.0                   plymouth 'cuda 340
18  21.0                        ford maverick
21  25.0                          peugeot 504
23  25.0                             saab 99e
26  10.0                            ford f250
27  10.0                            chevy c20
28  11.0                           dodge d200
32  25.0                        toyota corona
35  16.0            plymouth satellite custom
36  17.0            chevrolet chevelle malibu
39  14.0                     chevrolet impala
47  22.0                  chevrole

In [23]:
%%R
attach(Auto)
mean((mpg - predict(lm.fit, Auto))[-train]^2)

[1] 23.26601


In [24]:
pred = lm_fit.predict(auto_df.iloc[test_idx]['horsepower'])

In [25]:
auto_df.iloc[test_idx]['mpg'] - pred

2      1.710135
3      2.165254
4      0.165254
5     -0.531332
6      7.308871
         ...   
384    2.083584
385    8.083584
386    2.378907
390    7.003685
392    0.985733
Length: 196, dtype: float64

In [26]:
((auto_df.iloc[test_idx]['mpg'] - pred)**2).mean()

23.2660086465003

In [27]:
%%R
lm.fit2 <- lm(mpg ~ poly(horsepower, 2), data = Auto,
             subset = train)
mean((mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 18.71646


In [28]:
## http://davmre.github.io/blog/python/2013/12/15/orthogonal_poly

def ortho_poly_fit(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
            stop("'degree' must be less than number of unique points")
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q,r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)

    norm2 = np.sum(raw**2, axis=0)
    alpha = (np.sum((raw**2)*np.reshape(x,(-1,1)), axis=0)/norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z[:,1:], norm2, alpha

In [29]:
%%R
horsepower[sort(train)]

  [1] 130 225 150 225  95  95  97  88  46  90 113  90 193  88  90 100  88 100
 [19] 175 153 150 180 170 175 110  88  86  90  70  65  86 150 153 190 150 150
 [37]  76  87  69  86  88 175 150 137 150 158 150 100  46 150 167 170 180 100
 [55]  88  94  90 230  49  75 150 110 180 100 100  80  65 110 140 140  83  67
 [73]  52  75  93 105 150 110 105 110 110 129  83  96  97  70  95  86  83 150
 [91]  90  52  60 100  95  75 102 150 145 130 150  58  96  70 145 110 130 110
[109] 105 180 170 190  88  75  83  67  78  97 110  66  70  60 110  95 100  85
[127] 139 105  85  97  71 115  85  88  90 130 129 135 142 125  80  80  77  71
[145]  90  90 115  90  60  70  88  90  75  92  65  48  48  67  67  67  62 132
[163]  92  58  64  60  67  65  63  65  65  74  80 120  85  88  88  85  84  90
[181]  92  68  63  70  88  75  67  85  92 112  84  86  52  84  79  82


In [30]:
auto_df.iloc[train_idx-1]['horsepower']

1      130.0
9      225.0
13     150.0
14     225.0
15      95.0
       ...  
393     86.0
394     52.0
395     84.0
396     79.0
397     82.0
Name: horsepower, Length: 196, dtype: float64

In [31]:
%%R
poly(horsepower[sort(train)], 2)[61:70,]

                 1             2
 [1,]  0.089996747 -0.0423605989
 [2,]  0.014030501 -0.0661809634
 [3,]  0.146971431  0.0594979107
 [4,] -0.004961061 -0.0521376649
 [5,] -0.004961061 -0.0521376649
 [6,] -0.042944184 -0.0000530004
 [7,] -0.071431526  0.0600088070
 [8,]  0.014030501 -0.0661809634
 [9,]  0.071005185 -0.0603147238
[10,]  0.071005185 -0.0603147238


In [32]:
ortho_poly_fit(auto_df.iloc[train_idx-1]['horsepower'], 2)[0][60:70]

array([[ 8.99967465e-02, -4.23605989e-02],
       [ 1.40305005e-02, -6.61809634e-02],
       [ 1.46971431e-01,  5.94979107e-02],
       [-4.96106096e-03, -5.21376649e-02],
       [-4.96106096e-03, -5.21376649e-02],
       [-4.29441840e-02, -5.30004006e-05],
       [-7.14315262e-02,  6.00088070e-02],
       [ 1.40305005e-02, -6.61809634e-02],
       [ 7.10051850e-02, -6.03147238e-02],
       [ 7.10051850e-02, -6.03147238e-02]])

In [33]:
%%R
summary(lm.fit2)


Call:
lm(formula = mpg ~ poly(horsepower, 2), data = Auto, subset = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-12.8711  -2.6655  -0.0096   2.0806  16.1063 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)            23.5496     0.3175  74.182  < 2e-16 ***
poly(horsepower, 2)1 -123.5881     6.4587 -19.135  < 2e-16 ***
poly(horsepower, 2)2   47.7189     6.3613   7.501 2.25e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.439 on 193 degrees of freedom
Multiple R-squared:  0.705,	Adjusted R-squared:  0.702 
F-statistic: 230.6 on 2 and 193 DF,  p-value: < 2.2e-16



In [34]:
lm_fit2  = smf.ols(formula = 'mpg ~ ortho_poly_fit(horsepower, 2)[0]', data = auto_df.iloc[train_idx-1]).fit()

In [35]:
lm_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     230.6
Date:                Sat, 21 Jan 2023   Prob (F-statistic):           6.83e-52
Time:                        17:31:14   Log-Likelihood:                -568.72
No. Observations:                 196   AIC:                             1143.
Df Residuals:                     193   BIC:                             1153.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              23.8745      0.317     75.298      0.000      23.249      24.500
ortho_poly_fit(horsepower, 2)[0][0]   -89.3337      4.439    -20.125      0.000     -98.089     -80.579
ortho_poly_fit(horsepower, 2)[0][1]    33.2985      4.439      7.501      0.000      24.543      42.054
==============================================================================
Omnibus:                       11.485   Durbin-Watson:                   1.232
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.996
Skew:                           0.354   Prob(JB):                     0.000204
Kurtosis:                       4.257   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
auto_df_no_gaps = auto_df.copy(deep=True)
auto_df_no_gaps= auto_df_no_gaps.set_index(np.arange(1, auto_df_no_gaps.shape[0] + 1))

auto_train_mask_no_gaps = auto_df_no_gaps.index.isin(train_idx)

In [37]:
auto_df_no_gaps[auto_train_mask_no_gaps]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
9,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
13,15.0,8,400.0,150.0,3761,9.5,70,1,chevrolet monte carlo
14,14.0,8,455.0,225.0,3086,10.0,70,1,buick estate wagon (sw)
15,24.0,4,113.0,95.0,2372,15.0,70,3,toyota corona mark ii
...,...,...,...,...,...,...,...,...,...
388,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
389,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
390,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
391,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [38]:
auto_df_no_gaps.loc[train_idx]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
1,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
9,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina
13,15.0,8,400.0,150.0,3761,9.5,70,1,chevrolet monte carlo
14,14.0,8,455.0,225.0,3086,10.0,70,1,buick estate wagon (sw)
15,24.0,4,113.0,95.0,2372,15.0,70,3,toyota corona mark ii
...,...,...,...,...,...,...,...,...,...
388,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
389,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
390,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
391,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


In [39]:
%%R
Auto[sort(train),]

     mpg cylinders displacement horsepower weight acceleration year origin
1   18.0         8          307        130   3504         12.0   70      1
9   14.0         8          455        225   4425         10.0   70      1
13  15.0         8          400        150   3761          9.5   70      1
14  14.0         8          455        225   3086         10.0   70      1
15  24.0         4          113         95   2372         15.0   70      3
16  22.0         6          198         95   2833         15.5   70      1
17  18.0         6          199         97   2774         15.5   70      1
19  27.0         4           97         88   2130         14.5   70      3
20  26.0         4           97         46   1835         20.5   70      2
22  24.0         4          107         90   2430         14.5   70      2
24  26.0         4          121        113   2234         12.5   70      2
25  21.0         6          199         90   2648         15.0   70      1
29   9.0         8       

In [40]:
lm_model2  = smf.ols(formula = 'mpg ~ ortho_poly_fit(horsepower, 2)[0]', data = auto_df_no_gaps, subset=train_idx)
lm_fit2 = lm_model2.fit(method='qr')

In [41]:
lm_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     230.6
Date:                Sat, 21 Jan 2023   Prob (F-statistic):           6.83e-52
Time:                        17:31:16   Log-Likelihood:                -568.72
No. Observations:                 196   AIC:                             1143.
Df Residuals:                     193   BIC:                             1153.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              23.8745      0.317     75.298      0.000      23.249      24.500
ortho_poly_fit(horsepower, 2)[0][0]   -89.3337      4.439    -20.125      0.000     -98.089     -80.579
ortho_poly_fit(horsepower, 2)[0][1]    33.2985      4.439      7.501      0.000      24.543      42.054
==============================================================================
Omnibus:                       11.485   Durbin-Watson:                   1.232
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.996
Skew:                           0.354   Prob(JB):                     0.000204
Kurtosis:                       4.257   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [42]:
import sklearn.preprocessing

In [43]:
polynomial_features = sklearn.preprocessing.PolynomialFeatures(2, include_bias=False)

In [44]:
polynomial_features.fit_transform(np.array(auto_df_no_gaps['horsepower']).reshape(-1,1))

array([[1.3000e+02, 1.6900e+04],
       [1.6500e+02, 2.7225e+04],
       [1.5000e+02, 2.2500e+04],
       [1.5000e+02, 2.2500e+04],
       [1.4000e+02, 1.9600e+04],
       [1.9800e+02, 3.9204e+04],
       [2.2000e+02, 4.8400e+04],
       [2.1500e+02, 4.6225e+04],
       [2.2500e+02, 5.0625e+04],
       [1.9000e+02, 3.6100e+04],
       [1.7000e+02, 2.8900e+04],
       [1.6000e+02, 2.5600e+04],
       [1.5000e+02, 2.2500e+04],
       [2.2500e+02, 5.0625e+04],
       [9.5000e+01, 9.0250e+03],
       [9.5000e+01, 9.0250e+03],
       [9.7000e+01, 9.4090e+03],
       [8.5000e+01, 7.2250e+03],
       [8.8000e+01, 7.7440e+03],
       [4.6000e+01, 2.1160e+03],
       [8.7000e+01, 7.5690e+03],
       [9.0000e+01, 8.1000e+03],
       [9.5000e+01, 9.0250e+03],
       [1.1300e+02, 1.2769e+04],
       [9.0000e+01, 8.1000e+03],
       [2.1500e+02, 4.6225e+04],
       [2.0000e+02, 4.0000e+04],
       [2.1000e+02, 4.4100e+04],
       [1.9300e+02, 3.7249e+04],
       [8.8000e+01, 7.7440e+03],
       [9.

In [45]:
lm_model3  = smf.ols(formula = 'mpg ~ polynomial_features.fit_transform(np.array(horsepower).reshape(-1,1))', data = auto_df_no_gaps, subset=train_idx)
lm_fit3 = lm_model3.fit(method='qr')

In [46]:
lm_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     230.6
Date:                Sat, 21 Jan 2023   Prob (F-statistic):           6.83e-52
Time:                        17:31:17   Log-Likelihood:                -568.72
No. Observations:                 196   AIC:                             1143.
Df Residuals:                     193   BIC:                             1153.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=============================================================================================================================================
                                                                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                    58.8738      2.519     23.368      0.000      53.905      63.843
polynomial_features.fit_transform(np.array(horsepower).reshape(-1, 1))[0]    -0.4961      0.044    -11.192      0.000      -0.584      -0.409
polynomial_features.fit_transform(np.array(horsepower).reshape(-1, 1))[1]     0.0013      0.000      7.501      0.000       0.001       0.002
==============================================================================
Omnibus:                       11.485   Durbin-Watson:                   1.232
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.996
Skew:                           0.354   Prob(JB):                     0.000204
Kurtosis:                       4.257   Cond. No.                     1.21e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.21e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [47]:
%%R
lm.fit3 <- lm(mpg ~ poly(horsepower, 3), data=Auto, subset=train)
summary(lm.fit3)


Call:
lm(formula = mpg ~ poly(horsepower, 3), data = Auto, subset = train)

Residuals:
     Min       1Q   Median       3Q      Max 
-12.6625  -2.7108   0.0805   2.0724  16.1378 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)            23.5527     0.3185  73.946  < 2e-16 ***
poly(horsepower, 3)1 -123.6143     6.4755 -19.089  < 2e-16 ***
poly(horsepower, 3)2   47.8284     6.3935   7.481 2.58e-12 ***
poly(horsepower, 3)3    1.3825     5.8107   0.238    0.812    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.45 on 192 degrees of freedom
Multiple R-squared:  0.7051,	Adjusted R-squared:  0.7005 
F-statistic:   153 on 3 and 192 DF,  p-value: < 2.2e-16



In [48]:
lm_model3 = smf.ols(formula='mpg ~ ortho_poly_fit(horsepower, 3)[0]', data=auto_df_no_gaps, subset=train_idx)

lm_fit3 = lm_model3.fit()

lm_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     153.0
Date:                Sat, 21 Jan 2023   Prob (F-statistic):           1.14e-50
Time:                        17:31:17   Log-Likelihood:                -568.69
No. Observations:                 196   AIC:                             1145.
Df Residuals:                     192   BIC:                             1158.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              23.8745      0.318     75.114      0.000      23.248      24.501
ortho_poly_fit(horsepower, 3)[0][0]   -89.3337      4.450    -20.076      0.000     -98.111     -80.557
ortho_poly_fit(horsepower, 3)[0][1]    33.2985      4.450      7.483      0.000      24.522      42.075
ortho_poly_fit(horsepower, 3)[0][2]     1.0587      4.450      0.238      0.812      -7.718       9.836
==============================================================================
Omnibus:                       11.392   Durbin-Watson:                   1.228
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.312
Skew:                           0.365   Prob(JB):                     0.000287
Kurtosis:                       4.211   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
%%R
mean((mpg - predict(lm.fit3, Auto))[-train]^2)

[1] 18.79401


In [50]:
pred3 = lm_fit3.predict(auto_df_no_gaps[~auto_train_mask_no_gaps])

((pred3 - auto_df_no_gaps[~auto_train_mask_no_gaps]['mpg'])**2).mean()

19.743175316050696

In [51]:
%%R
set.seed(2)
train <- sample(392, 196)
lm.fit <- lm(mpg ~ horsepower, subset = train)
mean((mpg - predict(lm.fit, Auto))[-train]^2)

[1] 25.72651


In [52]:
data = robjects.r("""
library(ISLR2)
set.seed(2)
train <- sample(392, 196)
""")

new_train_idx = np.array(data)
new_train_idx = np.sort(new_train_idx)

new_auto_train_mask_no_gaps = auto_df_no_gaps.index.isin(new_train_idx)

In [53]:
lm_model = smf.ols(formula='mpg ~ horsepower', data = auto_df_no_gaps, subset = new_train_idx)

lm_fit = lm_model.fit()

pred = lm_fit.predict(auto_df_no_gaps[~new_auto_train_mask_no_gaps]['horsepower'])

((pred - auto_df_no_gaps[~new_auto_train_mask_no_gaps]['mpg'])**2).mean()

25.726510644813906

In [54]:
%%R
lm.fit2 <- lm(mpg ~ poly(horsepower, 2), data = Auto, subset = train)
mean((mpg - predict(lm.fit2, Auto))[-train]^2)

[1] 20.43036


In [55]:
lm_model2 = smf.ols(formula='mpg ~ ortho_poly_fit(horsepower, 2)[0]', data = auto_df_no_gaps, subset = new_train_idx)

lm_fit2 = lm_model2.fit()

pred2 = lm_fit2.predict(auto_df_no_gaps[~new_auto_train_mask_no_gaps]['horsepower'])

((pred2 - auto_df_no_gaps[~new_auto_train_mask_no_gaps]['mpg'])**2).mean()

20.432575508407442

In [56]:
%%R
lm.fit3 <- lm(mpg ~ poly(horsepower, 3), data = Auto, subset = train)
mean((mpg - predict(lm.fit3, Auto))[-train]^2)

[1] 20.38533


In [57]:
lm_model3 = smf.ols(formula='mpg ~ ortho_poly_fit(horsepower, 3)[0]', data = auto_df_no_gaps, subset = new_train_idx)

lm_fit3 = lm_model3.fit()

pred2 = lm_fit3.predict(auto_df_no_gaps[~new_auto_train_mask_no_gaps]['horsepower'])

((pred2 - auto_df_no_gaps[~new_auto_train_mask_no_gaps]['mpg'])**2).mean()

20.410400247238808

## 5.3.2 Leave-One-Out Cross-Validation

In [58]:
%%R
glm.fit <- glm(mpg ~ horsepower, data = Auto)
coef(glm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447 


In [59]:
glm_model = smf.glm(formula='mpg ~ horsepower', data=auto_df_no_gaps)
glm_fit = glm_model.fit()
glm_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64

In [60]:
%%R
lm.fit <- lm(mpg ~ horsepower, data = Auto)
coef(lm.fit)

(Intercept)  horsepower 
 39.9358610  -0.1578447 


In [61]:
lm_model = smf.ols(formula = 'mpg ~ horsepower', data=auto_df_no_gaps)
lm_fit = lm_model.fit()
lm_fit.params

Intercept     39.935861
horsepower    -0.157845
dtype: float64

In [62]:
%%R
library(boot)
glm.fit <- glm(mpg ~ horsepower, data = Auto)
cv.err <- cv.glm(Auto, glm.fit)
cv.err$delta

[1] 24.23151 24.23114


In [63]:
from sklearn.model_selection import cross_val_score, LeaveOneOut

In [64]:
glm_model = smf.glm(formula='mpg ~ horsepower', data=auto_df_no_gaps)
glm_fit = glm_model.fit()

In [65]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, formula, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
        self.formula = formula
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        data = pd.DataFrame(np.concatenate((X,y), axis=1), 
                            columns=['horsepower', 'mpg'])
        self.model_ = self.model_class(self.formula, data)
        self.results_ = self.model_.fit()
        return self
    def predict(self, X):
        horsepower = pd.DataFrame(X, columns = ['horsepower'])
        if self.fit_intercept:
            X = sm.add_constant(horsepower)
        return self.results_.predict(horsepower)

In [66]:
X = auto_df_no_gaps['horsepower'].values.reshape(-1,1)
y = auto_df_no_gaps['mpg'].values.reshape(-1,1)

In [67]:
## The main code for SMWrapper was taken from: https://stackoverflow.com/questions/41045752/using-statsmodel-estimations-with-scikit-learn-cross-validation-is-it-possible, however I adapted it slightly to be able to work with the smf.glm command.  The changes are as follows:
## 1) I had to create a self.formula attribute for use in creating the model in the fit method.

## 2) I had to concatenate X and y into a pandas df to be passed when creating the model in the fit method.

## 3) When creating the model, I have to pass self.formula from step 1) and the pandas df from step 2)

## 4) I had to convert X into a pandas df when making predictions using the predict method.

import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, formula, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
        self.formula = formula  ## 1)
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        data = pd.DataFrame(np.concatenate((X,y), axis=1), 
                            columns=['horsepower', 'mpg']) ## 2)
        self.model_ = self.model_class(self.formula, data) ## 3)
        self.results_ = self.model_.fit()
        return self
    def predict(self, X):
        horsepower = pd.DataFrame(X, columns = ['horsepower']) ##4)
        if self.fit_intercept:
            X = sm.add_constant(horsepower)
        return self.results_.predict(horsepower)


X = auto_df_no_gaps['horsepower'].values.reshape(-1,1)
y = auto_df_no_gaps['mpg'].values.reshape(-1,1)

loo = LeaveOneOut()


scores = cross_val_score(estimator = SMWrapper(smf.glm, 
                                      formula='mpg ~ horsepower',
                                      fit_intercept=False), 
                            X=X, y=y, cv = loo, scoring = 'neg_mean_squared_error', error_score='raise')

In [68]:
np.mean(np.abs(scores))

24.23151351792922

In [69]:
import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
        return self
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)

from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

X, y = make_regression(random_state=1, n_samples=300, noise=100)

print(cross_val_score(SMWrapper(sm.OLS), X, y, scoring='r2'))
print(cross_val_score(LinearRegression(), X, y, scoring='r2'))

[0.363997   0.28253087 0.3445757  0.50863318 0.6248952 ]
[0.363997   0.28253087 0.3445757  0.50863318 0.6248952 ]


In [70]:
## http://www.science.smith.edu/~jcrouser/SDS293/labs/lab7-py.html

X = auto_df_no_gaps['horsepower'].values.reshape(-1,1)
y = auto_df_no_gaps['mpg'].values.reshape(-1,1)

loo = LeaveOneOut()

skl_lm_model = sklearn.linear_model.LinearRegression()

scores = cross_val_score(skl_lm_model, X, y, cv = loo, scoring='neg_mean_squared_error')

np.mean(np.abs(scores))

24.231513517929226

In [71]:
%%R
cv.error <- rep(0, 10)
for (i in 1:10) {
    glm.fit <- glm(mpg ~ poly(horsepower, i), data = Auto)
    cv.error[i] <- cv.glm(Auto, glm.fit)$delta[1]
}
cv.error

 [1] 24.23151 19.24821 19.33498 19.42443 19.03321 18.97864 18.83305 18.96115
 [9] 19.06863 19.49093


In [72]:
def ortho_poly_predict(x, alpha, norm2, degree = 1):
    x = np.asarray(x).flatten()
    n = degree + 1
    Z = np.empty((len(x), n))
    Z[:,0] = 1
    if degree > 0:
        Z[:, 1] = x - alpha[0]
    if degree > 1:
      for i in np.arange(1,degree):
          Z[:, i+1] = (x - alpha[i]) * Z[:, i] - (norm2[i] / norm2[i-1]) * Z[:, i-1]
    Z /= np.sqrt(norm2)
    return Z[:,1:]

In [73]:
## As much as I'd like to cross_validate using the smf.glm function to replicate the R code, it's going to be difficult to iterate over different powers for the polynomial fit unless I can figure out a way to use the ortho_poly_fit() in the formula= parameter when calling to SMWrapper, otherwise I have to manually type out the formula 'mpg ~ hp1 + hp2 + ...' and also modify the SWMrapper class accordingly in the SWMrapper fit and predict methods...It's probably better to switch to sklearn at this point and using LinearRegression and PolynomialFeatures instead.

import statsmodels.api as sm
from sklearn.base import BaseEstimator, RegressorMixin

class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, formula, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
        self.formula = formula  ## 1)
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        X, self.norm2_, self.alpha_ = ortho_poly_fit(X, 2)
        
        
        data = pd.DataFrame(np.concatenate((X,y), axis=1), 
                            columns=['hp1', 'hp2', 'mpg']) ## 2)
        self.model_ = self.model_class(self.formula, data) ## 3)
        self.results_ = self.model_.fit()
        return self
    def predict(self, X):
        X = ortho_poly_predict(X, self.alpha_, self.norm2_, 2)
        horsepower = pd.DataFrame(X, columns = ['hp1', 'hp2']) ##4)
        if self.fit_intercept:
            X = sm.add_constant(horsepower)
        return self.results_.predict(horsepower)


X = auto_df_no_gaps['horsepower'].values.reshape(-1,1)
y = auto_df_no_gaps['mpg'].values.reshape(-1,1)

loo = LeaveOneOut()


scores = cross_val_score(estimator = SMWrapper(smf.glm, 
                                      formula='mpg ~ hp1 + hp2',
                                      fit_intercept=False), 
                            X=X, y=y, cv = loo, scoring = 'neg_mean_squared_error', error_score='raise')

In [74]:
np.mean(np.abs(scores))

19.24821312448969

In [75]:
cv_error = []

for i in range(1,11):
    polynomial_features = sklearn.preprocessing.PolynomialFeatures(i, include_bias=False)

    scores = cross_val_score(skl_lm_model, polynomial_features.fit_transform(X), y, cv = loo, scoring='neg_mean_squared_error')
    
    mean_score = np.mean(np.abs(scores))
    
    cv_error.append(mean_score)

In [76]:
cv_error

[24.231513517929226,
 19.24821312448967,
 19.33498406402931,
 19.42443031024277,
 19.03321248615882,
 18.97863406819667,
 19.129480449254846,
 19.224150660848743,
 19.133322843461364,
 18.93976572079586]

## 5.3.3 k-Fold Cross-Validation

In [77]:
%%R
set.seed(17)
cv.error.10 <- rep(0, 10)
for (i in 1:10) {
    glm.fit <- glm(mpg ~ poly(horsepower, i), data = Auto)
    cv.error.10[i] <- cv.glm(Auto, glm.fit, K = 10)$delta[1]
}
cv.error.10

 [1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520


In [78]:
cv_error_10 = []

for i in range(1,11):
    polynomial_features = sklearn.preprocessing.PolynomialFeatures(i, include_bias=False)

    scores = cross_val_score(skl_lm_model, polynomial_features.fit_transform(X), y, cv = 10, scoring='neg_mean_squared_error')
    
    mean_score = np.mean(np.abs(scores))
    
    cv_error_10.append(mean_score)

In [79]:
cv_error_10

[27.439933652339874,
 21.235840055802225,
 21.33660618322788,
 21.35388698195473,
 20.905641650770082,
 20.779180086179668,
 20.990939391569672,
 21.077615424551695,
 21.036905853431772,
 20.977623972381583]

## 5.3.4 The Bootstrap

### Estimating the Accuracy of a Statistic of Interest

In [80]:
%%R
alpha.fn <- function(data, index) {
    X <- data$X[index]
    Y <- data$Y[index]
    (var(Y) - cov(X, Y)) / (var(X) + var(Y) - 2 * cov(X, Y))
}

In [81]:
%%R
alpha.fn(Portfolio, 1:100)

[1] 0.5758321


In [82]:
pfolio_df = pd.read_csv("../../../datasets/Portfolio.csv")
pfolio_df = pfolio_df.set_index(np.arange(1, pfolio_df.shape[0] + 1))

In [83]:
def alpha_fn(data, index):
    X = data.loc[index]['X']
    Y = data.loc[index]['Y']
    var_x = np.var(X, ddof=1)
    var_y = np.var(Y, ddof=1)
    cov_x_y = np.cov(X, Y, ddof=1)[0][1]
    
    return (var_y - cov_x_y) / (var_x + var_y - 2 * cov_x_y)

In [84]:
alpha_fn(pfolio_df, np.arange(1,101))

0.5758320745928298

In [85]:
%%R
Portfolio$X[1:10]

 [1] -0.89525089 -1.56245433 -0.41708988  1.04435573 -0.31556841 -1.73712385
 [7]  1.96641316  2.15286790 -0.08120803 -0.89178179


In [86]:
pfolio_df.loc[1:10]['X']

1    -0.895251
2    -1.562454
3    -0.417090
4     1.044356
5    -0.315568
6    -1.737124
7     1.966413
8     2.152868
9    -0.081208
10   -0.891782
Name: X, dtype: float64

In [87]:
pfolio_df.head()

,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


In [88]:
X = pfolio_df[1:10]['X']
Y = pfolio_df[1:10]['Y']

In [89]:
%%R
var(Portfolio$X)

[1] 1.128642


In [90]:
np.var(pfolio_df['X'], ddof=1)

1.1286423581187572

In [91]:
pfolio_df['X'].var()

1.1286423581187572

In [92]:
%%R
var(Portfolio$Y)

[1] 1.308237


In [93]:
np.var(pfolio_df['Y'], ddof=1)

1.3082374688243965

In [94]:
pfolio_df['Y'].var()

1.3082374688243965

In [95]:
%%R
cov(Portfolio$X, Portfolio$Y)

[1] 0.6263583


In [96]:
np.cov(pfolio_df['X'], pfolio_df['Y'])[0][1]

0.6263582921063724

In [97]:
%%R
set.seed(7)
alpha.fn(Portfolio, sample(100, 100, replace=T))

[1] 0.5385326


In [98]:
%%R
set.seed(7)
sort(sample(100, 100, replace=T))

  [1]   2   3   4   5   6   6   8   8   8   8   9  10  11  12  12  15  16  16
 [19]  17  18  18  20  21  21  22  22  22  22  26  28  31  31  31  32  32  32
 [37]  33  34  34  36  38  38  38  39  40  40  40  41  41  42  43  43  44  44
 [55]  45  46  47  48  50  51  53  53  55  56  58  58  59  59  59  59  62  64
 [73]  66  67  67  68  72  72  72  73  74  75  77  78  79  80  81  83  87  87
 [91]  88  88  88  90  90  92  93  94  97 100


In [99]:
data = robjects.r("""
set.seed(7)
train <- sample(100, 100, replace=T)
""")

bstrap_idx = np.array(data)
bstrap_idx = np.sort(bstrap_idx)

In [100]:
alpha_fn(pfolio_df, bstrap_idx)

0.5385325919467925

In [101]:
%%R
boot(Portfolio, alpha.fn, R = 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.5758321 0.0007959475  0.08969074


In [102]:
alphas = []
n = pfolio_df.shape[0]
for _ in range(1000):
    idx = np.random.choice(np.arange(1, n+1), n)
    alpha = alpha_fn(pfolio_df, idx)
    alphas.append(alpha)
    
original_alpha = alpha_fn(pfolio_df, np.arange(1, n+1))
alpha_bstrap_mean = np.mean(alphas)
alpha_bstrap_std = np.std(alphas)

## Bias = bootstrap realization of the statistic - the original statistic from the original data
alpha_bias = alpha_bstrap_mean - original_alpha

print(f'Original Alpha: {original_alpha}')
print(f'Alpha Bias: {alpha_bias}')
print(f'Alpha Std: {alpha_bstrap_std}')

Original Alpha: 0.5758320745928298
Alpha Bias: 0.003201775211957636
Alpha Std: 0.08991920913525174


### Estimating the Accuracy of a Linear Regression Model

In [103]:
%%R
boot.fn <- function(data, index)
    coef(lm(mpg ~ horsepower, data = data, subset = index))
boot.fn(Auto, 1:392)

(Intercept)  horsepower 
 39.9358610  -0.1578447 


In [104]:
def boot_fn(data, index):
    model = smf.glm(formula = 'mpg ~ horsepower', data = data, subset = index)
    fit = model.fit()
    coefficients = fit.params
    
    return coefficients

In [105]:
boot_fn(auto_df_no_gaps, np.arange(1, 393))

Intercept     39.935861
horsepower    -0.157845
dtype: float64

In [106]:
%%R
set.seed(1)
boot.fn(Auto, sample(392, 392, replace = T))

(Intercept)  horsepower 
 40.3404517  -0.1634868 


In [107]:
data = robjects.r("""
set.seed(1)
samp <- sample(392, 392, replace=T)
""")

bstrap_idx = np.array(data)
bstrap_idx = np.sort(bstrap_idx)

In [108]:
boot_fn(auto_df_no_gaps, bstrap_idx)

Intercept     40.340452
horsepower    -0.163487
dtype: float64

In [109]:
%%R
boot.fn(Auto, sample(392, 392, replace = T))

(Intercept)  horsepower 
 40.1186906  -0.1577063 


In [110]:
idx = np.random.choice(np.arange(1,393), 392)
boot_fn(auto_df_no_gaps, idx)

Intercept     39.647563
horsepower    -0.156790
dtype: float64

In [111]:
%%R
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0544513229 0.841289790
t2* -0.1578447 -0.0006170901 0.007343073


In [112]:
%%R
myBootstrap <- boot(Auto, boot.fn, 1000)

head(myBootstrap$t)

         [,1]       [,2]
[1,] 41.08282 -0.1672521
[2,] 39.08086 -0.1529244
[3,] 39.86280 -0.1584696
[4,] 38.53024 -0.1472011
[5,] 41.37322 -0.1712221
[6,] 41.10563 -0.1665629


In [113]:
%%R
myBootstrap$t0

(Intercept)  horsepower 
 39.9358610  -0.1578447 


In [114]:
intercepts = []
slopes = []

n = auto_df_no_gaps.shape[0]

for _ in range(1000):
    idx = np.random.choice(np.arange(1, n+1), n)
    param = boot_fn(auto_df_no_gaps, idx)
    intercept = param[0]
    slope = param[1]
    intercepts.append(intercept)
    slopes.append(slope)

original_intercept = boot_fn(auto_df_no_gaps, np.arange(1, 393))[0]
original_slope = boot_fn(auto_df_no_gaps, np.arange(1, 393))[1]
intercept_bstrap_mean = np.mean(intercepts)
intercept_bstrap_std = np.std(intercepts)
slope_bstrap_mean = np.mean(slopes)
slope_bstrap_std = np.std(slopes)

## Bias = bootstrap realization of the statistic - the original statistic from the original data
## bias for intercepts
intercept_bias = intercept_bstrap_mean - original_intercept

## bias for slopes
slope_bias = slope_bstrap_mean - original_slope

print(f'Original Intercept: {original_intercept}')
print(f'Bstrap Intercept Bias: {intercept_bias}')
print(f'Bstrap Intercept Std: {intercept_bstrap_std}')
print(f'Original Slope: {original_slope}')
print(f'Bstrap Slope Bias: {slope_bias}')
print(f'Bstrap Slope Std: {slope_bstrap_std}')

Original Intercept: 39.93586102117048
Bstrap Intercept Bias: 0.08188595907797946
Bstrap Intercept Std: 0.8601742452149871
Original Slope: -0.15784473335365357
Bstrap Slope Bias: -0.0007708957600244815
Bstrap Slope Std: 0.007360089050082673


In [115]:
%%R
summary(lm(mpg ~ horsepower, data = Auto))$coef

              Estimate  Std. Error   t value      Pr(>|t|)
(Intercept) 39.9358610 0.717498656  55.65984 1.220362e-187
horsepower  -0.1578447 0.006445501 -24.48914  7.031989e-81


In [116]:
results = smf.ols(formula = 'mpg ~ horsepower', data = auto_df_no_gaps).fit().summary().tables[1]

In [117]:
## https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe

results_as_html = results.as_html()
results_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

results_df[['coef', 'std err', 't', 'P>|t|']]

,coef,std err,t,P>|t|
Intercept,39.9359,0.717,55.660,0.0
horsepower,-0.1578,0.006,-24.489,0.0


In [118]:
%%R
boot.fn <- function(data, index)
    coef(
      lm(mpg ~ horsepower + I(horsepower^2),
        data = data, subset = index)
    )

set.seed(1)
boot(Auto, boot.fn, 1000)


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164


In [119]:
## What does I do in the formula: https://stackoverflow.com/questions/24192428/what-does-the-capital-letter-i-in-r-linear-regression-formula-mean

def boot_fn(data, index):
    model = smf.ols(formula='mpg ~ horsepower + I(horsepower**2)', 
                    data=data, 
                    subset=index)
    fit = model.fit()
    params = fit.params
    
    return params

In [120]:
boot_fn(auto_df_no_gaps, np.arange(1, auto_df_no_gaps.shape[0]+1))

Intercept             56.900100
horsepower            -0.466190
I(horsepower ** 2)     0.001231
dtype: float64

In [121]:
intercepts = []
hp_1s = []
hp_2s = []

n = auto_df_no_gaps.shape[0]

for _ in range(1000):
    idx = np.random.choice(np.arange(1, n+1), n)
    param = boot_fn(auto_df_no_gaps, idx)
    intercept = param[0]
    hp_1 = param[1]
    hp_2 = param[2]
    intercepts.append(intercept)
    hp_1s.append(hp_1)
    hp_2s.append(hp_2)

original_intercept = boot_fn(auto_df_no_gaps, np.arange(1, 393))[0]
original_hp_1 = boot_fn(auto_df_no_gaps, np.arange(1, 393))[1]
original_hp_2 = boot_fn(auto_df_no_gaps, np.arange(1, 393))[2]

intercept_bstrap_mean = np.mean(intercepts)
intercept_bstrap_std = np.std(intercepts)

hp_1_bstrap_mean = np.mean(hp_1s)
hp_1_bstrap_std = np.std(hp_1s)

hp_2_bstrap_mean = np.mean(hp_2s)
hp_2_bstrap_std = np.std(hp_2s)

## Bias = bootstrap realization of the statistic - the original statistic from the original data
## bias for intercepts
intercept_bias = intercept_bstrap_mean - original_intercept

## bias for horsepower
hp_1_bias = hp_1_bstrap_mean - original_hp_1

## bias for horsepower**2
hp_2_bias = hp_2_bstrap_mean - original_hp_2

In [122]:
print(f'Original Intercept: {original_intercept}')
print(f'Bstrap Intercept Bias: {intercept_bias}')
print(f'Bstrap Intercept Std: {intercept_bstrap_std}')
print()
print(f'Original Slope: {original_hp_1}')
print(f'Bstrap horsepower Bias: {hp_1_bias}')
print(f'Bstrap horsepower Std: {hp_1_bstrap_std}')
print()
print(f'Original Slope: {original_hp_2}')
print(f'Bstrap horsepower**2 Bias: {hp_2_bias}')
print(f'Bstrap horsepower**2 Std: {hp_2_bstrap_std}')

Original Intercept: 56.90009970211517
Bstrap Intercept Bias: 0.04463179523038008
Bstrap Intercept Std: 2.0984610860675748

Original Slope: -0.46618962994736257
Bstrap horsepower Bias: -0.0010728975527891094
Bstrap horsepower Std: 0.03350097780158078

Original Slope: 0.0012305361007737656
Bstrap horsepower**2 Bias: 5.092998653565602e-06
Bstrap horsepower**2 Std: 0.00012099100390583751


In [123]:
%%R
summary(
    lm(mpg ~ horsepower + I(horsepower^2), data = Auto)
)$coef

                    Estimate   Std. Error   t value      Pr(>|t|)
(Intercept)     56.900099702 1.8004268063  31.60367 1.740911e-109
horsepower      -0.466189630 0.0311246171 -14.97816  2.289429e-40
I(horsepower^2)  0.001230536 0.0001220759  10.08009  2.196340e-21


In [124]:
model = smf.ols(formula = 'mpg ~ horsepower + I(horsepower**2)',
                data = auto_df_no_gaps)
fit = model.fit()

results = fit.summary().tables[1]

In [125]:
results_as_html = results.as_html()
results_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

results_df[['coef', 'std err', 't', 'P>|t|']]

,coef,std err,t,P>|t|
Intercept,56.9001,1.800,31.604,0.0
horsepower,-0.4662,0.031,-14.978,0.0
I(horsepower ** 2),0.0012,0.000,10.080,0.0


# The End

# Extra stuff

In [ ]:
type(glm_model)

In [ ]:
import glmnet_python
from glmnet import glmnet; from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict

In [ ]:
import statsmodels.api as sm

In [ ]:
X = sm.add_constant(auto_df_no_gaps['horsepower'].astype(np.float64).values)

glm_fit = glmnet(x = X, y = auto_df_no_gaps['mpg'].astype(np.float64).values, family='gaussian')

In [ ]:
glmnetCoef(glm_fit)

In [ ]:
polynomial_features.fit_transform(np.array(auto_df['horsepower'][0:4]).reshape(-1, 1))[:,1:]

In [ ]:
%%R
horsepower[1:3]

In [ ]:
def poly(x, p):
    x = np.array(x)
    X = np.transpose(np.vstack((x**k for k in range(p+1))))
    return np.linalg.qr(X)[0][:,1:]

In [ ]:
poly(auto_df['horsepower'][0:4], 2)

In [ ]:
auto_df[['horsepower_poly_1', 'horsepower_poly_2'] 

In [ ]:
lm_model2 = smf.ols(formula = 'mpg ~ poly(horsepower,2)', data = auto_df.iloc[train_idx-1])
lm_fit2 = lm_model2.fit()
pred = lm_fit2.predict(auto_df.iloc[test_idx]['horsepower'])
((auto_df.iloc[test_idx]['mpg'] - pred)**2).mean()

In [ ]:
lm_model2 = smf.ols(formula = 'mpg ~ poly(horsepower, 2)', data = auto_df.iloc[train_idx-1])

In [ ]:
lm_fit2 = lm_model2.fit()

In [ ]:
pred = lm_fit2.predict(auto_df.iloc[test_idx]['horsepower'])
((auto_df.iloc[test_idx]['mpg'] - pred)**2).mean()

In [ ]:
lm_fit2.predict(auto_df['horsepower'][1])

In [ ]:
lm_fit2.summary()

In [ ]:
auto_df['horsepower'][0]

In [ ]:
lm_model2 = smf.ols(formula = 'mpg ~ polynomial_features.fit_transform(np.array(auto_df.iloc[train_idx-1]["horsepower"]).reshape(-1,1))[:,1:]', data = auto_df.iloc[train_idx-1])

lm_fit2 = lm_model2.fit()

# poly.fit_transform(np.array(auto_df['horsepower'][0:4]).reshape(-1, 1))

In [ ]:
lm_fit2.summary()

In [ ]:
polynomial_features.fit_transform(np.array(auto_df.iloc[train_idx-1]["horsepower"]).reshape(-1,1))[:,1:]

In [ ]:
def ortho_poly_fit(x, degree = 1):
    n = degree + 1
    x = np.asarray(x).flatten()
    if(degree >= len(np.unique(x))):
            stop("'degree' must be less than number of unique points")
    xbar = np.mean(x)
    x = x - xbar
    X = np.fliplr(np.vander(x, n))
    q,r = np.linalg.qr(X)

    z = np.diag(np.diag(r))
    raw = np.dot(q, z)

    norm2 = np.sum(raw**2, axis=0)
    alpha = (np.sum((raw**2)*np.reshape(x,(-1,1)), axis=0)/norm2 + xbar)[:degree]
    Z = raw / np.sqrt(norm2)
    return Z[:,1:]##, norm2, alpha

In [ ]:
X = ortho_poly_fit(auto_df['horsepower'][0:20], 2)
X

In [ ]:
lm_model2 = smf.ols(formula = 'mpg ~ ortho_poly_fit(horsepower, 2)', data = auto_df.iloc[train_idx-1])

lm_fit2 = lm_model2.fit(method='qr')

# poly.fit_transform(np.array(auto_df['horsepower'][0:4]).reshape(-1, 1))

In [ ]:
lm_fit2.summary()

In [ ]:
ortho_poly_fit(auto_df.iloc[train_idx - 1]['horsepower'], 2)

In [ ]:
X = ortho_poly_fit(auto_df.iloc[train_idx-1]['horsepower'], 2)

In [ ]:
X

In [ ]:
X_df = pd.DataFrame(data={'h1':X[:,0], 'h2':X[:,1], 'mpg': auto_df['horsepower']})

In [ ]:


lm_model2 = smf.ols(formula = 'mpg ~ h1 + h2', data = X_df.iloc[train_idx-1])

lm_fit2 = lm_model2.fit()
lm_fit2.summary()
# poly.fit_transform(np.array(auto_df['horsepower'][0:4]).reshape(-1, 1))

In [ ]:
auto_df.set_index(1, auto_df.shape[0]+1)

In [ ]:
test = auto_df.set_index(np.arange(1,auto_df.shape[0]+1))

In [ ]:
test.loc[train_idx]

In [ ]:
lm_model2 = smf.ols(formula = 'mpg ~ ortho_poly_fit(horsepower, 2)', data = test, subset=train_idx)

lm_fit2 = lm_model2.fit(method='qr')

lm_fit2.summary()

In [ ]:
lm_fit2.fittedvalues

In [ ]:
auto_df.iloc[train_idx-1]['mpg']

In [ ]:
ortho_poly_fit(auto_df.iloc[train_idx-1]['horsepower'], 2)